# ANN with 6 strain rates

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use("mime")
import h5py
from scipy.optimize import curve_fit
import lmfit
import os

colors = ['#bb0000', '#00bb00', "#0000bb", '#bbbb00', '#bb00bb', "#00bbbb", '#bbbbbb', '#770000', '#007700', "#000077", '#777700', '#770077', "#007777", '#777777', '#440000', '#004400', "#000044", '#444400', '#440044', "#0044444", '#444444','#000000']

In [ ]:
baseSize = (8, 6)  # Base size of a subplot

def sbPlot(n):
    if (n == 1): return 1, 1
    if (n <= 2): return 1, 2
    if (n <= 4): return 2, 2
    if (n <= 6): return 3, 2
    if (n <= 9): return 3, 3
    if (n <= 12): return 4, 3
    return 0, 0

def sbPlotSize(n):
    x, y = sbPlot(n)
    return baseSize[0] * y, baseSize[1] * x

In [ ]:
h5f = h5py.File('../GleebleData.h5','r')
allData = h5f['all'][:]
shortData = h5f['short'][:]
h5f.close()

In [ ]:
allData.shape, shortData.shape

Remove first point of each curve, where $\varepsilon^p=0$

In [ ]:
identData0 = allData[allData[:,0] != 0]
identData = allData[allData[:,0] != 0]
#identData = shortData[shortData[:,0]!=0]

# Identification of the ANN parameters


In [ ]:
# Importation de TensorFlow
import tensorflow as tf
from keras.models import Sequential
from keras.metrics import MeanSquaredError
from keras.layers.core import Dense, Activation
from keras import optimizers
tf.keras.backend.set_floatx('float64')

In [ ]:
training = False

In [ ]:
epspl = np.unique(identData[:,0])
depspl = np.unique(identData[:,1])
Tl = np.unique(identData[:,2])
nEpsp = depspl.shape[0]

T0 = Tl[0]
epsLogBase = depspl[0]
Tm = 1460

In [ ]:
identData[:,1] = np.log(identData[:,1] / epsLogBase)

In [ ]:
minEntries = identData.min(axis=0)
maxEntries = identData.max(axis=0)
rangeEntries = maxEntries - minEntries
NNentries = (identData - minEntries) / rangeEntries

In [ ]:
print("Max error for normalized data is : %5.4E" %(NNentries * rangeEntries + minEntries - identData).max())

In [ ]:
plt.figure(figsize = sbPlotSize(nEpsp))
plt.rc('text', usetex = True)
plt.subplots_adjust(hspace = 0.3)
idx = 1
for epspv, epspvv in zip(np.unique(identData[:,1]), depspl):
    xs, ys = sbPlot(nEpsp)
    plt.subplot(xs, ys, idx)
    cl = 0
    for T, TOrig in zip(np.unique(identData[:,2]), Tl):
        # filter on T
        data = NNentries[(identData[:,2]==T) & (identData[:,1]==epspv), :]
        # Plot the curves
        plt.plot(data[:,0]*rangeEntries[0]+minEntries[0], data[:,3]*rangeEntries[3]+minEntries[3], label=r'$T=' + str(TOrig) + '^{\circ}C$', linewidth = 3)
        cl += 1
    plt.legend(loc = 'lower right',fancybox = True, numpoints = 1, fontsize = 10)
    plt.xlabel(r'$Plastic\ strain\ \varepsilon^{p}$', fontsize = 16)
    plt.ylabel(r'$Von\ Mises\ stress\ \sigma$', fontsize = 16)
    plt.title(r'$\dot{\varepsilon^p}=' + str(epspvv) + '\ s^{-1}$', fontsize = 16)
    idx += 1
plt.show()

In [ ]:
colEps = 0
colEpsp = 1
colT = 2
colSig = 3
NNinput = NNentries[:,colEps:colSig]
NNoutput = NNentries[:,colSig]
NNinput.shape, NNoutput.shape

In [ ]:
if training:
    models = []  # The list of models to use
    convergenceDatas = []   # To store convergence curves
    
    # FACT = ['tanh', 'sigmoid']
    # COUCH = [5, 7, 9, [5,3], [7,4], [9,5]]
    
    FACT = ['sigmoid']
    COUCH = [[7, 5],[9, 5],[9, 7],[13, 7],[15, 7]]
    
    for f in FACT:
        for c in COUCH:
            desc = '3'
            model = Sequential()
            if type(c) == list:
                fst = True
                for k in c:
                    if (fst): model.add(Dense(k, input_dim = 3, activation = f))
                    else: model.add(Dense(k, activation = f))
                    fst = False
                    desc += '-' + str(k)
            else:
                model.add(Dense(c, input_dim = 3, activation = f))
                desc += '-' + str(c)
            model.add(Dense(1))
            desc += '-1-' + f
            model._name = desc
            models.append(model)
            convergenceDatas.append([desc, np.array([])])

In [ ]:
if training:
    for model in models:
        model.compile(loss='mean_squared_error', optimizer='adam')
        model.summary()

In [ ]:
def saveInternalMatrices(ANN, filename):
    if len(ANN.layers) == 1: 
        np.savez(filename,
                 logBase = np.array([epsLogBase]),
                 minEntries = minEntries, 
                 maxEntries = maxEntries,
                 w1 = ANN.layers[0].get_weights()[0].T, 
                 b1 = ANN.layers[0].get_weights()[1].reshape(len(ANN.layers[0].get_weights()[1]),1))
    if len(ANN.layers) == 2: 
        np.savez(filename, 
                 logBase = np.array([epsLogBase]),
                 minEntries = minEntries, 
                 maxEntries = maxEntries,
                 w1 = ANN.layers[0].get_weights()[0].T, 
                 b1 = ANN.layers[0].get_weights()[1].reshape(len(ANN.layers[0].get_weights()[1]),1),
                 w2 = ANN.layers[1].get_weights()[0].T, 
                 b2 = ANN.layers[1].get_weights()[1].reshape(len(ANN.layers[1].get_weights()[1]),1))
    if len(ANN.layers) == 3: 
        np.savez(filename, 
                 logBase = np.array([epsLogBase]),
                 minEntries = minEntries, 
                 maxEntries = maxEntries,
                 w1 = ANN.layers[0].get_weights()[0].T, 
                 b1 = ANN.layers[0].get_weights()[1].reshape(len(ANN.layers[0].get_weights()[1]),1),
                 w2 = ANN.layers[1].get_weights()[0].T, 
                 b2 = ANN.layers[1].get_weights()[1].reshape(len(ANN.layers[1].get_weights()[1]),1),
                 w3 = ANN.layers[2].get_weights()[0].T, 
                 b3 = ANN.layers[2].get_weights()[1].reshape(len(ANN.layers[2].get_weights()[1]),1)) 
    if len(ANN.layers) == 4: 
        np.savez(filename, 
                 logBase = np.array([epsLogBase]),
                 minEntries = minEntries, 
                 maxEntries = maxEntries,
                 w1 = ANN.layers[0].get_weights()[0].T, 
                 b1 = ANN.layers[0].get_weights()[1].reshape(len(ANN.layers[0].get_weights()[1]),1),
                 w2 = ANN.layers[1].get_weights()[0].T, 
                 b2 = ANN.layers[1].get_weights()[1].reshape(len(ANN.layers[1].get_weights()[1]),1),
                 w3 = ANN.layers[2].get_weights()[0].T, 
                 b3 = ANN.layers[2].get_weights()[1].reshape(len(ANN.layers[2].get_weights()[1]),1),
                 w4 = ANN.layers[3].get_weights()[0].T, 
                 b4 = ANN.layers[3].get_weights()[1].reshape(len(ANN.layers[3].get_weights()[1]),1))     

In [ ]:
dataPath = 'ANN-6'

iterationsNumber = 50 # Define the number of iteration to do
epochNumber = 100     # Define the number of epoch for each iteration
subSave = True

if training:
    for model, convergenceData in zip (models, convergenceDatas):
        print("MODEL :", model.name)
        if (subSave):
            try:
                os.mkdir(dataPath + '/' + model.name)
            except:
                pass
        for i in range(iterationsNumber):
            history = model.fit(NNinput, NNoutput, epochs = epochNumber, verbose = 0, shuffle = True)
            loss = history.history['loss']
            convergenceData[1] = np.append(convergenceData[1], loss)
            if (subSave):
                saveInternalMatrices(model, dataPath + '/' + model.name + '/ANN-' + str(i))
            saveInternalMatrices(model, dataPath + '/' + model.name + '.npz')
    
            print("Iteration :", i + 1, "/", iterationsNumber, '-> %8.6E' % loss[-1], "  ", end = '\r')
        print("\n")

In [ ]:
if training:
    for model in models:
        model.save(dataPath + '/' + model.name + '/model')

In [ ]:
loadModels = ['3-13-7-1-sigmoid', '3-15-7-1-sigmoid', '3-7-5-1-sigmoid', '3-9-5-1-sigmoid','3-9-7-1-sigmoid']

In [ ]:
if (training==False):
    models = []              # The list of models to use
    for modName in loadModels:
        new_model = tf.keras.models.load_model(dataPath + '/' + modName + '/model')
        models.append(new_model)

In [ ]:
if training:
    for convergenceData in convergenceDatas:
        name = convergenceData[0]
        convCurve = convergenceData[1]
        np.savez(dataPath + '/CD-' + name, convCurve = convCurve)
        print("Convergence data %s saved" % (name))

In [ ]:
if (training==False):
    convergenceDatas = []   # To store convergence curves
    for name in loadModels:
        NN = np.load(dataPath + '/CD-' + name + '.npz')
        cc = [name, NN['convCurve']]
        convergenceDatas.append(cc)

In [ ]:
plt.figure(figsize = (12, 9))
plt.rc('text', usetex = True)
cl = 0
for convergenceData in convergenceDatas:
    name = convergenceData[0]
    curve = convergenceData[1]
    shortCurve = curve[::10]
    x = np.linspace(0, len(curve), len(shortCurve))
    plt.plot(x, np.log10(shortCurve), label = name, color=colors[cl], linewidth = 3)
    cl += 1
#plt.grid()
plt.xlabel(r'Iteration', fontsize = 16)
plt.ylim(-5.25, -3.5)
#plt.xlim(0, 1000)

plt.ylabel(r'Quadratic Error $\log_{10}(\Delta E)$', fontsize = 16)
plt.title(r'Global convergence of the Neural Network', fontsize = 16)
plt.legend(loc = 'upper right',fancybox = True, numpoints = 1, fontsize = 14)
plt.savefig("Convergence.svg")
plt.show()

In [ ]:
model = models[0]
model.name

In [ ]:
# Plot the curves
from matplotlib.lines import Line2D
def create_dummy_line(**kwds):
    return Line2D([], [], **kwds)

for model in models:
    plt.figure(figsize = sbPlotSize(nEpsp))
    plt.rc('text', usetex = True)
    idx = 1
    plt.subplots_adjust(hspace = 0.3)
    for epsp in list(depspl):
        xs, ys = sbPlot(nEpsp)
        plt.subplot(xs, ys, idx)
        sbdata = shortData[shortData[:,1]==epsp]
        cl =0
        for temp in list(Tl):
            sbdata1 = sbdata[sbdata[:,2]==temp]
            plt.plot(sbdata1[:,0], sbdata1[:,3], colors[cl], marker = 's', markersize = 5, linestyle = 'none')
            inp = np.zeros((epspl.shape[0],3))
            inp[:,0] = (epspl - minEntries[0]) / rangeEntries[0]
            inp[:,1] = (np.log(epsp / epsLogBase) - minEntries[1]) / rangeEntries[1]
            inp[:,2]  = (temp - minEntries[2]) / rangeEntries[2]
            plt.plot(epspl, model.predict(inp)*rangeEntries[3]+minEntries[3], colors[cl], linewidth = 2.5)
            plt.rcParams['xtick.labelsize'] = 16
            plt.rcParams['ytick.labelsize'] = 16
            cl +=1
        plt.xlim(0, 0.7)
        plt.ylim(bottom=0)
        plt.xlabel(r'strain $\varepsilon$', fontsize = 16) # Labels the x axis
        plt.ylabel(r'flow stress $\sigma^y$ (MPa)', fontsize = 16) # Labels the y axis
        plt.title(r'strain rate $\dot{\varepsilon} = ' + str(epsp) + '$ s$^{-1}$', fontsize = 16) # Self explicit command
        idx += 1
        
    legendLines = []
    cl = 0
    for temp in list(Tl):
        legendLines.append((r'$T=$' + str(int(temp)) + r'$^{\circ}$C', {'color':colors[cl], 'linestyle':'-', 'linewidth':2.5, 'marker':'s'}))
        cl += 1
    
    plt.legend([create_dummy_line(**l[1]) for l in legendLines],[l[0] for l in legendLines], 
               loc = 'upper center', fontsize = 12, ncols = 5, bbox_to_anchor = (0.0, -0.2), shadow = False)
    
    plt.savefig("CompExpANN-"+model.name+".svg")
    plt.show()

In [ ]:
for model in models:
    ARstress = (model.predict(NNinput)*rangeEntries[3]+minEntries[3]).flatten()
    EAAR = np.sum(np.abs((identData0[:,3] - ARstress)/(identData0[:,3])))*100/ARstress.shape[0]
    RMSE = np.sqrt(np.sum((identData0[:,3] - ARstress)**2)/ARstress.shape[0])
    print("Model %s" %(model.name))
    print('  RMSE = %g' %(RMSE)+' MPa')
    print("  EAAR = %g" %(EAAR) + ' %')

In [ ]:
for model in models:
    ARstress = (model.predict(NNinput)*rangeEntries[3]+minEntries[3])
    data = np.concatenate((identData0[:,0:3],ARstress),axis=1)
    h5f = h5py.File(dataPath+'/'+model.name+'.h5','w')
    h5f.create_dataset('data', data = data)
    h5f.close()